<a href="https://colab.research.google.com/github/rudradeep22/Tweet_Decoder/blob/main/Experiment_sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Final Task: Aim to classify the tweets.

In [ ]:
! pip install tensorflow_text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 58.7 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from sklearn.model_selection import train_test_split
from sklearn import metrics
import matplotlib.pyplot as plt
import tensorflow_text as text

In [ ]:
!gdown --id 1RARLRRVmkaoHQV05pfGIvyBH6XRjcSr9

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1RARLRRVmkaoHQV05pfGIvyBH6XRjcSr9
To: /content/tweets.csv
100% 8.70M/8.70M [00:00<00:00, 56.5MB/s]


In [ ]:
data= pd.read_csv('tweets.csv')
data

,Unnamed: 0,tweet,target
0,0,ndtv anchor let go over to aap live press conf...,0
1,1,sambit patra attacks rahul gandhi over his cow...,0
2,2,crediting the bjp with taking development to s...,1
3,3,ktrbrs do do false allegations on iasassociati...,0
4,4,gujarat assembly yet to get rs cr as gst compe...,1
...,...,...,...
49472,25683,1ddrive modispeakstonews18 loksabhaelections20...,1
49473,25684,i am seriously worried about future of indian ...,1
49474,25685,if narendramodi is so bad then why does incind...,0
49475,25686,on this 23rd march pakistan day i urge all ind...,0


Government tweets are labeled with 1 while opposition tweets are labeled with 0. I have already done the data pre-processing part to make it easy for you all.
Go-ahead and do **encode -> spliting -> classification using model**.
You have already done this task in last two assignments. **Best of Luck!**

Optional: You can also try clustring algorithem, this would give you a better result.

In [ ]:
X, y = data['tweet'], data['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)
X_train.head(5)

39969    indian politics is interesting pm is political...
37440    disruptionofcorruption modi govt efforts in la...
39615    rahuigandh narendramodi i thought he got engag...
20285    modi name modi person modi symbol modi symbol ...
29986    farooq abdullah has accused prime minister nar...
Name: tweet, dtype: object

In [ ]:
def build_classifier_model():
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
  preprocessing_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3", name='preprocessing')
  encoder_inputs = preprocessing_layer(text_input)
  encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4", trainable=True, name='BERT_encoder')
  outputs = encoder(encoder_inputs)
  net = outputs['pooled_output']
  net = tf.keras.layers.Dropout(0.1)(net)
  net = tf.keras.layers.Dense(1, activation=None, name='classifier')(net)
  return tf.keras.Model(text_input, net)

In [ ]:
classifier_model = build_classifier_model()
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
metrics = tf.metrics.BinaryAccuracy()
epochs = 5
init_lr = 3e-5
opt = tf.keras.optimizers.Adam(learning_rate=3e-5)

In [ ]:
classifier_model.compile(optimizer=opt,
                         loss=loss,
                         metrics=metrics)

In [ ]:
device_name = tf.test.gpu_device_name()
device_name

'/device:GPU:0'

In [ ]:
with tf.device(device_name):
  history = classifier_model.fit(X_train, y_train,validation_data=(X_test, y_test), epochs=5, batch_size = 50)

Epoch 1/5
 47/743 [>.............................] - ETA: 15:19 - loss: 0.5895 - binary_accuracy: 0.6485

KeyboardInterrupt: ignored

### Using KMeans Clustering to classify tweets

In [ ]:
text = []
for review in X_train:
  text.append(review)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

In [ ]:
cv = TfidfVectorizer()
classify_text = cv.fit_transform(text)

In [ ]:
k=2
model = KMeans(n_clusters=2, init='k-means++', n_init=1)
model.fit(classify_text)


KMeans(n_clusters=2, n_init=1)

In [ ]:
test_text = []
for review in X_test:
  test_text.append(review)

In [ ]:
classify_test = cv.transform(test_text)
pred = model.predict(classify_test)
truth = list(y_test)
total = 0
correct = 0
for prediction, value in zip(pred,truth):
  total+=1
  if prediction == value:
    correct+=1
accuracy = correct/total * 100
accuracy

58.96523848019402

### Using Naive Bayes Classifier to classify

In [ ]:
from gensim.models import Word2Vec
from sklearn.naive_bayes import MultinomialNB

In [ ]:
tokenized_sentences = [sentence.lower().split() for sentence in text]
model = Word2Vec(tokenized_sentences, window=5, min_count=1)

In [ ]:
sentence_embeddings = []
for sentence in tokenized_sentences:
    embeddings = [model.wv[word] for word in sentence if word in model.wv]
    if embeddings:
        avg_embedding = sum(embeddings) / len(embeddings)
        sentence_embeddings.append(avg_embedding)
    else:
        sentence_embeddings.append([])

X_train_word2vec = sentence_embeddings
X_train_word2vec = np.array(sentence_embeddings)
X_train_word2vec = X_train_word2vec - np.min(X_train_word2vec)

X_train_word2vec = X_train_word2vec / np.max(X_train_word2vec)

In [ ]:
classifier = MultinomialNB()
classifier.fit(X_train_word2vec, y_train)

In [ ]:
new_tokenized_sentences = [sentence.lower().split() for sentence in test_text]
new_sentence_embeddings = []
for sentence in new_tokenized_sentences:
    embeddings = [model.wv[word] for word in sentence if word in model.wv]
    if embeddings:
        avg_embedding = np.mean(embeddings, axis=0)
        new_sentence_embeddings.append(avg_embedding)
    else:
        new_sentence_embeddings.append([])

X_test_word2vec = np.array(new_sentence_embeddings)
X_test_word2vec = X_test_word2vec - np.min(X_test_word2vec)
X_test_word2vec = X_test_word2vec / np.max(X_test_word2vec)

In [ ]:
pred = classifier.predict(X_test_word2vec)
truth = list(y_test)
accuracy = metrics.accuracy_score(truth, pred)
print(accuracy)

0.5484236054971706
0.5484236054971706
